### Data Collections Complete, Now Transitioning to Processing Data into Tables

#### Creating category & genre id and name reference table

In [ ]:
import pandas as pd
import ast

def extract_id_name_table(df, column_name, id_col, name_col):
    """
    column_name: name of the column to parse
    id_col: name of the output ID column
    name_col: name of the output name column
    """
    parsed_col = column_name + '_parsed'
    temp_df = df[[column_name]].copy()
    temp_df[parsed_col] = temp_df[column_name].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    
    id_name_set = set()
    for row in temp_df[parsed_col]:
        if isinstance(row, list): # handle for rows with NA
            for item in row:
                id_name_set.add((item['id'], item['description']))
    
    id_name_table = pd.DataFrame(list(id_name_set), columns=[id_col, name_col])
    id_name_table[id_col] = id_name_table[id_col].astype(int)
    id_name_table = id_name_table.sort_values(by=id_col).reset_index(drop=True)
    
    return id_name_table

In [ ]:
df = pd.read_csv("steam_top_games_with_price_logs_and_game_info.csv")
cat_table = extract_id_name_table(df, 'categories', 'category_id', 'category_name')
genre_table = extract_id_name_table(df, 'genres', 'genre_id', 'genre_name')

cat_table.to_csv("table_category.csv", index=False)
genre_table.to_csv("table_genre.csv", index=False)

In [ ]:
cat_table

#### Cleaning price log column into seperate table

In [ ]:
df = pd.read_csv("steam_top_games_with_price_logs_and_game_info.csv")
df2 = df[['steam_id', 'price_logs']].copy()
df2.head()

In [ ]:
import ast
import pandas as pd

all_logs = []

for _, row in df2.iterrows():
    game_id = row.steam_id
    try:
        price_logs = ast.literal_eval(row.price_logs)
        all_logs.extend([
            {'steam_id': game_id, 'date': log['timestamp'], 'price': log['deal']['price']['amount'], 'cut_pct': log['deal']['cut'], 'regular': log['deal']['regular']['amount']}
            for log in price_logs
        ])
    except Exception as e:
        print(f"Error processing row with steam_id={game_id}: {e}")


price_history_df = pd.DataFrame(all_logs)
# change column type. note: contains mixed time zone
price_history_df['date'] = pd.to_datetime(price_history_df['date'], utc=True)

price_history_df['date_only'] = price_history_df['date'].dt.date  # for comparing by calendar day

# Remove 0-price entries only when duplicate date with non-zero price exists
# Count entries with same steam_id and date_only
duplicate_groups = price_history_df.groupby(['steam_id', 'date_only']).size().reset_index(name='count')
duplicate_dates = duplicate_groups[duplicate_groups['count'] > 1][['steam_id', 'date_only']]
price_history_df = price_history_df.merge(duplicate_dates, on=['steam_id', 'date_only'], how='left', indicator=True)

# Remove rows with price == 0 AND duplicate date
price_history_df = price_history_df[~((price_history_df['price'] == 0) & (price_history_df['_merge'] == 'both'))]
price_history_df.drop(columns=['_merge'], inplace=True)

# Remove all identical rows
price_history_df = price_history_df.drop_duplicates(subset=['steam_id', 'date_only', 'price', 'cut_pct','regular'], keep='first')
price_history_df.drop(columns=['date_only'], inplace=True)

# clean duplicate entry with no price change. (take the record with smallest date for duplicate entries)
price_history_df = price_history_df.sort_values(by=['steam_id', 'date'], ascending=[True, True])

price_history_df_clean = (
    price_history_df
    .groupby('steam_id', group_keys=False)[['steam_id', 'date', 'price', 'cut_pct','regular']]
    .apply(lambda group: group.loc[
        (group[['price', 'cut_pct', 'regular']].shift() != group[['price', 'cut_pct', 'regular']]).any(axis=1)
    ])
)

price_history_df_clean.to_csv("table_price_log.csv", index=False)
price_history_df_clean

In [ ]:
# Load the CSV
df = pd.read_csv('table_price_log.csv')

# Ensure the 'date' column is in datetime format
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Define the years to process
years = range(2015, 2026)

for year in years:
    start_date = pd.to_datetime(f"{year}-01-01", utc=True)
    end_date = pd.to_datetime(f"{year + 1}-01-01", utc=True)
    
    filtered_df = df[(df['date'] >= start_date) & (df['date'] < end_date)]
    filename = f"table_price_log_{year}.csv"
    
    filtered_df.to_csv(filename, index=False)
    
    print(f"{len(filtered_df)} rows for {year}. New file: {filename}")


### Create link table between category/genre versus game

In [ ]:
import pandas as pd
import ast

# Load your big DataFrame
df = pd.read_csv("steam_top_games_with_price_logs_and_game_info.csv")[['steam_id', 'categories', 'genres']]

# Helper: Convert string to list of dicts
def parse_list_of_dicts(x):
    try:
        return ast.literal_eval(x) if isinstance(x, str) else x
    except:
        return []

# Parse categories and genres
df['categories'] = df['categories'].apply(parse_list_of_dicts)
df['genres'] = df['genres'].apply(parse_list_of_dicts)

# Build category-game link table
category_links = []
for _, row in df.iterrows():
    steam_id = row['steam_id']
    if isinstance(row['categories'], list):  # Check if valid list
        for cat in row['categories']:
            cat_id = int(cat['id']) if 'id' in cat else None
            if cat_id is not None:
                category_links.append({'steam_id': steam_id, 'category_id': cat_id})

category_link_df = pd.DataFrame(category_links).sort_values(by=['steam_id']).reset_index(drop=True)

# Build genre-game link table
genre_links = []
for _, row in df.iterrows():
    steam_id = row['steam_id']
    if isinstance(row['genres'], list):  # Check if valid list
        for genre in row['genres']:
            genre_id = int(genre['id']) if 'id' in genre else None
            if genre_id is not None:
                genre_links.append({'steam_id': steam_id, 'genre_id': genre_id})

genre_link_df = pd.DataFrame(genre_links).sort_values(by=['steam_id']).reset_index(drop=True)

# Save or print
print("Category-Game Link Table:\n", category_link_df.head())
print("\nGenre-Game Link Table:\n", genre_link_df.head())

category_link_df.to_csv("link_game_category.csv", index=False)
genre_link_df.to_csv("link_game_genre.csv", index=False)

#### Creating publisher & developer id and name reference table

In [ ]:
import pandas as pd
import ast  # for safely evaluating string lists

# Load your DataFrame
df = pd.read_csv("steam_top_games_with_price_logs_and_game_info.csv")[['publishers', 'developers']]

# Initialize sets to collect unique publishers and developers
all_publishers = set()
all_developers = set()

# Step 1: Flatten all publisher names
for pub_str in df['publishers']:
    try:
        pub_list = ast.literal_eval(pub_str) if isinstance(pub_str, str) else pub_str
        if isinstance(pub_list, list):
            all_publishers.update(pub_list)
    except Exception as e:
        print(f"Skipping a problematic publisher entry: {e}")

# Step 2: Flatten all developer names
for dev_str in df['developers']:
    try:
        dev_list = ast.literal_eval(dev_str) if isinstance(dev_str, str) else dev_str
        if isinstance(dev_list, list):
            all_developers.update(dev_list)
    except Exception as e:
        print(f"Skipping a problematic developer entry: {e}")

# Step 3: Create publisher_id table
publisher_table = pd.DataFrame({
    'publisher_id': range(1, len(all_publishers)+1),
    'publisher_name': list(all_publishers)
})

# Step 4: Create developer_id table
developer_table = pd.DataFrame({
    'developer_id': range(1, len(all_developers)+1),
    'developer_name': list(all_developers)
})

# Optional: Sort for easier reading
publisher_table = publisher_table.sort_values('publisher_id').reset_index(drop=True)
developer_table = developer_table.sort_values('developer_id').reset_index(drop=True)

# Print or save
print("Publisher Table:\n", publisher_table)
print("\nDeveloper Table:\n", developer_table)

# Save as CSV
publisher_table.to_csv("table_publisher.csv", index=False)
developer_table.to_csv("table_developer.csv", index=False)

### Create link table between publisher/developer versus game

In [ ]:
import pandas as pd
import ast

# Load your main DataFrame
df = pd.read_csv("steam_top_games_with_price_logs_and_game_info.csv")

# Load reference tables
publisher_ref = pd.read_csv("table_publisher.csv")
developer_ref = pd.read_csv("table_developer.csv")

# Helper: Convert string lists to actual lists
def safe_parse_list(x):
    try:
        return ast.literal_eval(x) if isinstance(x, str) else x
    except:
        return []

# Parse publisher and developer columns
df['publishers'] = df['publishers'].apply(safe_parse_list)
df['developers'] = df['developers'].apply(safe_parse_list)

# Step 1: Build publisher-game link table
publisher_links = []
for _, row in df.iterrows():
    steam_id = row['steam_id']
    for pub in row['publishers']:
        pub_id = publisher_ref.loc[publisher_ref['publisher_name'] == pub, 'publisher_id']
        if not pub_id.empty:
            publisher_links.append({'steam_id': steam_id, 'publisher_id': pub_id.values[0]})

publisher_link_df = pd.DataFrame(publisher_links)

# Step 2: Build developer-game link table
developer_links = []
for _, row in df.iterrows():
    steam_id = row['steam_id']
    for dev in row['developers']:
        dev_id = developer_ref.loc[developer_ref['developer_name'] == dev, 'developer_id']
        if not dev_id.empty:
            developer_links.append({'steam_id': steam_id, 'developer_id': dev_id.values[0]})

developer_link_df = pd.DataFrame(developer_links)

# Sort by steam_id before saving
publisher_link_df = publisher_link_df.sort_values(by=['steam_id']).reset_index(drop=True)
developer_link_df = developer_link_df.sort_values(by=['steam_id']).reset_index(drop=True)

# Save or preview
print("Publisher-Game Link Table:\n", publisher_link_df.head())
print("\nDeveloper-Game Link Table:\n", developer_link_df.head())

# Save as CSV
publisher_link_df.to_csv("link_game_publisher.csv", index=False)
developer_link_df.to_csv("link_game_developer.csv", index=False)

#### Creating platform id and name reference table

In [ ]:
platform = pd.DataFrame({
    'platform_id': [1, 2, 3],
    'platform_name': ['Windows', 'Mac', 'Linux']
})

platform.to_csv("table_platform.csv", index=False)

### Create link table between platform versus game

In [ ]:
# Read your CSV
df = pd.read_csv('steam_top_games_with_price_logs_and_game_info.csv')  # columns: steam_id, is_windows, is_mac, is_linux

# Melt to long format
melted = df.melt(id_vars='steam_id', 
                 value_vars=['is_windows', 'is_mac', 'is_linux'],
                 var_name='platform_name_raw',
                 value_name='has_platform')

# Keep only rows where value is 1 (has platform)
melted = melted[melted['has_platform'] == 1]

# Map to platform_id
platform_map = {
    'is_windows': 1,
    'is_mac': 2,
    'is_linux': 3
}
melted['platform_id'] = melted['platform_name_raw'].map(platform_map)

# Final game_platform table
game_platform_df = melted[['steam_id', 'platform_id']]
game_platform_df.to_csv("link_game_platform.csv", index=False)

### Create game table

In [ ]:
# Load full DataFrame
df = pd.read_csv("steam_top_games_with_price_logs_and_game_info.csv")

# Keep only the specified columns
columns_to_keep = ['steam_id','itad_id','game','link','release_dates','header_image']

filtered_df = df[columns_to_keep]

# Optionally print or save
print(filtered_df.head())
filtered_df.to_csv("table_game.csv", index=False)